In [7]:
import json
import os
import shutil
import sqlite3
from datetime import datetime
from time import sleep
#import query
import logzero
import requests
#import secret
import numpy as np
from logzero import logger
from requests import get
import calendar
import regex as re
import pandas as pd

from collections import Counter
import itertools
from itertools import cycle, islice
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
#import query
from scipy.cluster.hierarchy import ward, dendrogram
from sklearn import cluster, datasets
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA, NMF
from sklearn.manifold import TSNE, MDS
from sklearn.metrics import accuracy_score, silhouette_samples, silhouette_score
from sklearn.metrics.cluster import completeness_score, homogeneity_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PolynomialFeatures
import sqlite3
import seaborn as sns
from pandas.tseries.offsets import MonthEnd

In [8]:
import warnings

import pandas as pd
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [9]:
#Gathering full list of US zip codes

all_zips = []

# open file and read the content in a list
with open('../data/uszips.txt', 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        all_zips.append(current_place)

In [10]:
#Gathering zip codes which have necessary data for installation

data_zips = []

# open file and read the content in a list
with open('../data/datazips.txt', 'r') as filehandle:
    filecontents = filehandle.readlines()

    for line in filecontents:
        # remove linebreak which is the last character of the string
        current_place = line[:-1]

        # add item to the list
        data_zips.append(current_place)
        
data_zips = sorted(data_zips)

In [11]:
percent_zips = len(data_zips)/len(all_zips)
percent_zips

0.4470492396813903

In [12]:
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

In [13]:
def zipdata(zipcode):
    code = search.by_zipcode(zipcode)
    code = code.to_dict()
    #print(code)
    income = code["median_household_income"]
    value = code["median_home_value"]
    density = code["population_density"]
    land_area = code["land_area_in_sqmi"]
    houses = code["housing_units"]
    state = code["state"]
    return income, value, density, houses, land_area, state

In [14]:
zip_df = pd.DataFrame({'zipcode':data_zips})
zip_df['income'],zip_df['value'],zip_df['density'],zip_df['houses'],zip_df['land'], zip_df['state'] = zip(*zip_df["zipcode"].map(zipdata))


In [15]:
from rasterstats import zonal_stats
stats = zonal_stats("../plotly_zips.geojson", "../data/nsrdb_full_disc_2019_dni_scaled.tif")
irradiance = [f['mean'] for f in stats]

ParseException: Unknown type: '../PLOTLY_ZIPS.GEOJSON'


ValueError: Can't parse ../plotly_zips.geojson as a geojson Feature object

In [ ]:
import json
with open("../plotly_zips.geojson") as f:
    zips = json.load(f)

In [ ]:
zip_ids = [i['id'] for i in zips['features']]

irr_map = dict(zip(zip_ids,irradiance))

zip_df['irradiance'] = zip_df.zipcode.map(irr_map)
zip_df.head()

In [ ]:
prices = pd.read_excel("../data/avgprice_annual.xlsx", header=1)

prices_res_2019 = prices[(prices["Industry Sector Category"] == "Total Electric Industry") & (prices.Year == 2019)][['State','Residential']]
price_map = dict(zip(prices_res_2019.State, prices_res_2019.Residential))

zip_df['e_price'] = zip_df.state.map(price_map)
zip_df.head()

In [ ]:
electricians = pd.read_csv("../data/PolicyMap Data 2021-07-09 0319UTC.csv")
electricians = electricians.rename(columns = {"Average annual wage in the specialty trade contractors industry ":"Avg_wage"})
electricians_2 = electricians.copy()
electricians = electricians[['FIPS Code','Avg_wage']].dropna().astype(int)
electricians['FIPS Code'] = electricians['FIPS Code'].astype(str).apply(lambda n: n.zfill(5))
electricians.head()

In [ ]:
zip_county = pd.read_excel("../data/ZIP_COUNTY_032021.xlsx", header=0)
zip_county = zip_county[["ZIP","COUNTY"]].astype(str)

zip_county['ZIP'] = zip_county.ZIP.apply(lambda n: n.zfill(5))
zip_county['COUNTY'] = zip_county.COUNTY.apply(lambda n: n.zfill(5))

z_c = dict(zip(zip_county.ZIP,zip_county.COUNTY))
c_w = dict(zip(electricians['FIPS Code'],electricians.Avg_wage))

zip_df['county'] = zip_df.zipcode.map(z_c)
zip_df['avg_contractor_wage'] = zip_df.county.map(c_w)
zip_df.head()

In [ ]:
zip_cluster_df = zip_df[['zipcode','income','value','density',
                         'houses','land','irradiance',
                         'e_price','avg_contractor_wage']].rename(
    columns = {'income':'med_income','value':'med_home_value',
               'density':'pop_density','houses':'home_count',
               'land':'land_area','irradiance': 'NREL_PSM_2019', 
               'e_price': 'cost_electricity'})
zip_cluster_df[['med_income', 'med_home_value', 
                'pop_density', 'home_count',
                'land_area', 'NREL_PSM_2019', 
                'cost_electricity','avg_contractor_wage']] = zip_cluster_df[['med_income', 'med_home_value', 
                                                                             'pop_density', 'home_count',
                                                                             'land_area', 'NREL_PSM_2019', 
                                                                             'cost_electricity','avg_contractor_wage']].astype(float)
zip_cluster_df.head()

In [ ]:
zip_cluster_df_full = zip_cluster_df.dropna()

In [ ]:
# import plotly.express as px

# fig = px.choropleth_mapbox(zip_cluster_df_full, geojson=zips, locations='zipcode', color='NREL_PSM_2019',
#                            color_continuous_scale="Viridis",
#                            mapbox_style="carto-positron",
#                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
#                            opacity=0.5,
#                            labels={'NREL_PSM_2019':'PSM'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
zip_cluster_df_full['housing_density'] = zip_cluster_df_full.home_count.values/zip_cluster_df_full.land_area.values

In [ ]:
#sns.violinplot(data=bk_P[['system_size_DC']])

In [ ]:
#sns.violinplot(data=bk_P_2[['system_size_DC']])

In [ ]:
#sns.violinplot(data=bk_P_3[['system_size_DC']])

In [ ]:
#sns.lineplot(data=costs_df, x="install_month", y="cost_per_kW")

In [ ]:
#sns.lineplot(data=costs_df2, x="install_month", y="cost_per_kW")

In [ ]:
#sns.lineplot(data=rebate_df, x="install_month", y="rebate_or_grant")

In [ ]:
with open('../data/sub_25.p', 'rb') as fp:
    sub_25_map = pickle.load(fp)

In [ ]:
zip_cluster_df_full['install_count'] = zip_cluster_df_full.zipcode.map(sub_25_map)
zip_cluster_df_full['percent_homes'] = zip_cluster_df_full.install_count.values/zip_cluster_df_full.home_count.values
zip_cluster_df_full.head()

In [ ]:
# import plotly.express as px

# fig = px.choropleth_mapbox(zip_cluster_df_full, geojson=zips, locations='zipcode', color='percent_homes',
#                            color_continuous_scale="Viridis",
#                            mapbox_style="carto-positron",
#                            zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
#                            opacity=0.5,
#                            labels={'system_size_DC':'kW'}
#                           )
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.show()

In [ ]:
zc_pair = zip_cluster_df_full[['med_income', 
                               'med_home_value', 
                               'pop_density', 
                               'NREL_PSM_2019', 
                               'cost_electricity', 
                               'avg_contractor_wage',
                               'percent_homes']].dropna()

#zc_pair['cap_per_home'] = pd.cut(zc_pair.cap_per_home, bins = 5)
#zc_pair.head()
sns.pairplot(data = zc_pair)

In [ ]:
from sklearn.linear_model import LinearRegression

X = zc_pair.drop(columns = 'percent_homes')
y = zc_pair.percent_homes

model = LinearRegression()
# fit the model
model.fit(X, y)
# get importance
importance = model.coef_
importance

In [ ]:
zc_pair['prediction'] = model.predict(X)
zc_pair['performance_vs_expectation'] = (zc_pair.percent_homes - zc_pair.prediction) / zc_pair.prediction
zc_pair['zipcode'] = zip_cluster_df_full.zipcode

zc_pair = zc_pair[~zc_pair['performance_vs_expectation'].isin(find_anomalies(zc_pair['performance_vs_expectation']))]
zc_pair.head()

In [ ]:
fig = px.choropleth_mapbox(zc_pair, geojson=zips, locations='zipcode', color='performance_vs_expectation',
                           color_continuous_scale="Viridis",
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'system_size_DC':'kW'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
zc_pair.head()

In [ ]:
import pickle

with open('../data/earliest_dates.p', 'rb') as fp:
    earliest_dict = pickle.load(fp)

In [ ]:
plt.clf()
fig, axes = plt.subplots(25,figsize = (12, 120))
min_yr = pd.to_datetime('1998-04-30 00:00:00').year
min_mo = pd.to_datetime('1998-04-30 00:00:00').month




for i, key in enumerate(earliest_dict.keys()):
    eval_df =  bk_P_rt[bk_P_rt.zip_code.isin(earliest_dict[key].keys())]
    eval_df['p_mo'] = eval_df.zip_code.map(earliest_dict[key])
    
    eval_df['num_months'] = eval_df.install_month.apply(
        lambda d: (d.year - min_yr) * 12 + (d.month - min_mo)
    )
    
    eval_df['program_months'] = eval_df.num_months - eval_df.p_mo
    
    eval_df['total_cost'] = eval_df.total_installed_price - eval_df.rebate_or_grant
    #eval_df['prog_date'] = pd.to_datetime(eval_df.zip_code.map(program_dict[key]))
    #eval_df['num_months'] = (eval_df.install_month.dt.year - eval_df['prog_date'].dt.year) * 12 + (eval_df.install_month.dt.month - eval_df['prog_date'].dt.month)
    
    plotter = eval_df.groupby(['p_mo','program_months']).total_cost.count().reset_index()
    #display(plotter.sort_values(by = 'num_months').head(20))
    #plotter['prog_date'] = plotter['prog_date'].apply(lambda c: str(c))
    maxes = plotter.groupby(['p_mo']).total_cost.max().reset_index()
    max_dict = dict(zip(maxes.p_mo,maxes.total_cost))
    plotter['max_val'] = plotter.p_mo.map(max_dict)
    plotter['percent_max'] = plotter.total_cost / plotter.max_val
    plotter_2 = plotter.groupby('program_months').percent_max.mean().reset_index()
    sns.lineplot(ax=axes[i], data=plotter_2, x='program_months', y='percent_max')  
    axes[i].set_title("program_id_{}".format(str(key)))
    

In [ ]:
zip_cluster_df_full['install_level'] = pd.qcut(zip_cluster_df_full.percent_homes, 7,
                                              labels=[0,1,2,3,4,5,6]) #"low", "med-low", "med", "med-high","high"
zip_cluster_df_full = zip_cluster_df_full.dropna()


In [ ]:
# scale_pipe = Pipeline(
#     [
        
#         ("standard",StandardScaler()),
#     ]
# )

# df_raw = zip_cluster_df_full.copy()[['housing_density','med_income','NREL_PSM_2019', 'cost_electricity']]
# df_transformed = scale_pipe.fit_transform(df_raw)

# h_scores = {}
# c_scores = {}



# y_WS = zip_cluster_df_full.install_level
# # reduced hyperparameters for runtime savings - derived from wider range
# for i in np.linspace(.2,.5,16):
#     for j in np.linspace(5,50,10):
#         try:
#             dbscan = DBSCAN(eps = i, min_samples = j)
#             cls = dbscan.fit_predict(df_transformed)
#             hs = homogeneity_score(cls, y_WS)
#             cs = completeness_score(cls, y_WS)
#             if len(np.unique(cls)) > 1:
#                 h_scores[(i,j)] = hs
#                 c_scores[(i,j)] = cs
#             else:
#                 h_scores[(i,j)] = 0
#                 c_scores[(i,j)] = 0
#         except:
#             h_scores[(i,j)] = 0
#             c_scores[(i,j)] = 0
# sdh = sorted(h_scores.items(), key=lambda item: item[1], reverse = True)
# sdc = sorted(c_scores.items(), key=lambda item: item[1], reverse = True)
    
    
# sdc

In [ ]:
# df_raw = zip_cluster_df_full.copy()[['housing_density','med_income','NREL_PSM_2019', 'cost_electricity']]
# df_transformed = scale_pipe.fit_transform(df_raw)


# i_c = zip_cluster_df_full.install_level

# agg = DBSCAN(eps = .22, min_samples = 5) #AgglomerativeClustering(n_clusters = 11)
# cls_a2 = agg.fit_predict(df_transformed)

# pca_a = df_transformed.copy()
# tsne_a = df_transformed.copy()

# X_pca_a = PCA(n_components = 2, random_state=42).fit_transform(pca_a)
# X_tsne_a = TSNE(n_components = 2, random_state = 42).fit_transform(tsne_a)

# plt.clf()
# fig, axes = plt.subplots(4, figsize=(12,12))

# axes[0].scatter(X_pca_a[:, 0], X_pca_a[:, 1], s=10, c=cls_a2)
# axes[1].scatter(X_tsne_a[:, 0], X_tsne_a[:, 1], s=10, c=cls_a2)
# axes[2].scatter(X_pca_a[:, 0], X_pca_a[:, 1], s=10, c=i_c)
# axes[3].scatter(X_tsne_a[:, 0], X_tsne_a[:, 1], s=10, c=i_c)



# axes[0].set_title("PCA_Agglomerative")
# axes[1].set_title("tsne_Agglomerative")
# axes[0].set_title("PCA_lvl")
# axes[1].set_title("tsne_lvl")
        
# cs = completeness_score(cls_a2, i_c)
# print(cs)

In [ ]:
state_list = ['CA','OR','WA','AZ','NM','TX','FL','NY','MA', 'WI','PA']